In [1]:
import re, statistics
from pathlib import Path
import pandas as pd

ROW_RE = re.compile(r"^([A-Za-z0-9 .:_-]+?)\s+(-?\d+(?:\.\d+)?)\s+(-?\d+(?:\.\d+)?)\s+(-?\d+(?:\.\d+)?)\s+(-?\d+(?:\.\d+)?)\s+(-?\d+(?:\.\d+)?)\s+(\d+)\s*$")

def parse_pref_file(path: Path) -> pd.DataFrame:
    text = path.read_text(encoding='utf-8', errors='ignore')
    lines = text.splitlines()
    rows = []
    run_idx = None
    capture = False
    for line in lines:
        if line.strip().startswith('==========') and 'Performance Summary (ms)' in line:
            m = re.search(r"==========\s*(\d+)\s*Performance", line)
            run_idx = int(m.group(1)) if m else None
            capture = True
            continue
        if capture:
            if not line.strip():
                capture = False
                continue
            if line.strip().startswith('name'):
                continue
            m = ROW_RE.match(line)
            if m:
                name = m.group(1).strip()
                vals = [float(m.group(i)) for i in range(2,7)]
                cnt = int(m.group(7))
                rows.append({'run': run_idx,'metric': name,'mean': vals[0],'median': vals[1],'rmse': vals[2],'min': vals[3],'max': vals[4],'count': cnt})
    return pd.DataFrame(rows)

def aggregate_one(df: pd.DataFrame, metric_name: str) -> dict:
    sub = df[df['metric'] == metric_name]
    if sub.empty:
        return {'metric': metric_name}
    return {
        'metric': metric_name,
        'mean': sub['mean'].mean(),
        'median': sub['median'].mean(),
        'rmse': sub['rmse'].mean(),
        'min': sub['min'].mean(),
        'max': sub['max'].mean(),
        'count_avg': sub['count'].mean(),
    }

def aggregate_extracts(df: pd.DataFrame) -> pd.DataFrame:
    ex = df[df['metric'].str.contains('Extract', na=False)]
    out_rows = []
    for name, g in ex.groupby('metric'):
        out_rows.append({
            'metric': name,
            'mean': g['mean'].mean(),
            'median': g['median'].mean(),
            'rmse': g['rmse'].mean(),
            'min': g['min'].mean(),
            'max': g['max'].mean(),
            'count_avg': g['count'].mean(),
        })
    return pd.DataFrame(out_rows).sort_values('metric')

def analyze_performance(folders, labels, base_dir='./Performance'):
    assert len(folders) == len(labels)
    main_rows, extract_rows = [], []
    for folder, lab in zip(folders, labels):
        folder = str(folder)
        pref = Path(base_dir) / folder / f"{folder}_pref.txt"
        if not pref.exists():
            pref = Path(base_dir) / folder / f"{folder.lower()}_pref.txt"
        if not pref.exists():
            print(f"[WARN] pref not found: {pref}")
            continue
        df = parse_pref_file(pref)
        for m in ['Tracking','Total Init','Pipeline','Correlation']:
            agg = aggregate_one(df, m)
            agg.update({'exp': folder, 'label': ' + '.join(lab)})
            main_rows.append(agg)
        edf = aggregate_extracts(df)
        for _, r in edf.iterrows():
            extract_rows.append({
                'exp': folder, 'label': ' + '.join(lab), 'extractor': r['metric'],
                'mean': r['mean'], 'median': r['median'], 'rmse': r['rmse'],
                'min': r['min'], 'max': r['max'], 'count_avg': r['count_avg']
            })
    return pd.DataFrame(main_rows), pd.DataFrame(extract_rows)


In [9]:
folders = ['fr1_xyz_akz', 'fr1_xyz_orb', 'fr1_xyz_sft']
labels  = [['AKAZE'], ['ORB'], ['SIFT']]
main_df, extracts_df = analyze_performance(folders, labels, base_dir='./Performances/')
main_df.head(20)

,metric,mean,median,rmse,min,max,count_avg,exp,label
0,Tracking,4.07135,4.24865,0.92190,1.790,6.948,773.00,fr1_xyz_akz,AKAZE
1,Total Init,1245.10040,1245.10040,0.00000,1200.263,1294.748,1.00,fr1_xyz_akz,AKAZE
2,Pipeline,23.55310,22.81025,4.78320,18.255,150.655,798.00,fr1_xyz_akz,AKAZE
3,Correlation,0.00000,0.00000,0.00020,0.000,0.074,773.00,fr1_xyz_akz,AKAZE
4,Tracking,3.22265,3.25690,0.50700,0.897,5.141,618.70,fr1_xyz_orb,ORB
5,Total Init,6631.63775,6631.63775,0.00000,1664.659,9634.771,2.95,fr1_xyz_orb,ORB
6,Pipeline,10.42610,9.58280,2.93440,6.182,67.147,798.00,fr1_xyz_orb,ORB
7,Correlation,0.00000,0.00000,0.00000,0.000,0.011,618.70,fr1_xyz_orb,ORB
8,Tracking,4.06950,4.10745,0.85595,1.371,8.069,772.00,fr1_xyz_sft,SIFT
9,Total Init,1849.05205,1849.05205,0.00000,1818.418,1897.049,1.00,fr1_xyz_sft,SIFT


In [10]:
extracts_df

,exp,label,extractor,mean,median,rmse,min,max,count_avg
0,fr1_xyz_akz,AKAZE,AKAZE Extract,17.7315,17.6324,1.03365,14.546,33.566,798.0
1,fr1_xyz_orb,ORB,ORB Extract,5.3474,5.2812,0.57945,3.891,10.053,798.0
2,fr1_xyz_sft,SIFT,SIFT Extract,20.5404,20.4249,2.78440,16.033,47.091,798.0


In [4]:
from pathlib import Path
out_dir = Path('/mnt/data/perf_analysis')
out_dir.mkdir(parents=True, exist_ok=True)
main_path = out_dir / 'performance_main.csv'
ext_path  = out_dir / 'performance_extracts.csv'
main_df.to_csv(main_path, index=False)
extracts_df.to_csv(ext_path, index=False)
print('Saved to:', main_path, 'and', ext_path)

PermissionError: [Errno 13] Permission denied: '/mnt/data'